In [2]:
from fastapi import FastAPI, File, Form, UploadFile, HTTPException
from typing import List, Annotated
import asyncio
import random
import tempfile
import shutil
import os
import fitz
import io
import base64
import datetime
from PIL import Image
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# Azure AI Document Intelligence
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
# Load environment variables
from dotenv import load_dotenv
load_dotenv()
# Custom Utils
from customutils import *

In [30]:
import pandas as pd

businessLine = 'FBI'

# DEFINE PATH
pdfpath = f"versionAchive/V0.9-alpha/testFiles/{businessLine}/"
# LIST ALL PDF
lspdf = [f for f in os.listdir(pdfpath) if f.endswith('.pdf')]
# DF
dfPDF = pd.DataFrame()
dfPDF['FULL_PATH'] = [pdfpath+f for f in lspdf]
dfPDF['FILE_NAME'] = lspdf
dfPDF['RESPONSE'] = None
dfPDF.head()

,FULL_PATH,FILE_NAME,RESPONSE
0,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003031-Actimalt Liquid Regular.pdf,None
1,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003047-8. FAMB279 BROWN SUGAR TYPE FL.pdf,None
2,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003729-4. Acerola JC_TH.pdf,None
3,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003890-ACCELERZYME CPG.pdf,None
4,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000007436-Specification_1101103010101_Apple...,None


In [ ]:
for i in range(len(dfPDF)):
    try:
        full_path = dfPDF['FULL_PATH'][i]
        file_name = dfPDF['FILE_NAME'][i]
        url = "http://127.0.0.1:8000/v1_get_products_and_suppliers"
        input_secret = os.getenv('CUSTOM_SECRET1')
        files = [('inputListDocumentation', open(full_path, 'rb'))]
        data = {'inputSecret': input_secret}
        response = requests.post(url, files=files, data=data)
        dfPDF['RESPONSE'].iat[i] = response.json()['products_and_suppliers']
        print('SUCCESS ', file_name)
    except:
        print('ERROR   ', file_name)
dfPDF.head()

SUCCESS  PIM000003031-Actimalt Liquid Regular.pdf
SUCCESS  PIM000003047-8. FAMB279 BROWN SUGAR TYPE FL.pdf
SUCCESS  PIM000003729-4. Acerola JC_TH.pdf
SUCCESS  PIM000003890-ACCELERZYME CPG.pdf
SUCCESS  PIM000007436-Specification_1101103010101_Apple juice concentrate_Austria Juice Standard - Stock quality - Medium acidity.pdf
SUCCESS  PIM000008862-EXTER 100 TDS 2021.pdf
SUCCESS  PIM000009406-Rokoagar LS ORGANIC - Food Applications Eng - E01.pdf
SUCCESS  PIM000009441-SPEC APPLE FLAVOUR NPC1917M.pdf
SUCCESS  PIM000009983-YU15 rev02 - organic chickpeas flour.pdf
SUCCESS  PIM000021685-Specification Apple JC 35209000710000 20180717.pdf
SUCCESS  PIM000039886-973772999 DPS Apple Pomace fine milled .pdf
SUCCESS  PIM000041915-K110 rev08 - Q.C. Organic kamut.pdf
SUCCESS  PIM000042350-Spec-AD Flat Parsley Medium, Large.pdf
SUCCESS  PIM000047605-702469-001(20221018)FD Cranberry Powder 50mesh-DKSH AU.pdf
SUCCESS  PIM000048276-RSF-200 - Prod Info rev. 3 (1).pdf
SUCCESS  PIM000088058-LACIN101-E-202101.

,FULL_PATH,FILE_NAME,RESPONSE
0,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003031-Actimalt Liquid Regular.pdf,"[{'PRODUCT_NAME': 'ACTIMALT LIQUID REGULAR', '..."
1,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003047-8. FAMB279 BROWN SUGAR TYPE FL.pdf,[{'PRODUCT_NAME': 'FAMB279 N&A BROWN SUGAR TYP...
2,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003729-4. Acerola JC_TH.pdf,"[{'PRODUCT_NAME': 'Acerola juice concentrate, ..."
3,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003890-ACCELERZYME CPG.pdf,"[{'PRODUCT_NAME': 'Accelerzyme CPG BF', 'SUPPL..."
4,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000007436-Specification_1101103010101_Apple...,"[{'PRODUCT_NAME': 'Apple juice concentrate', '..."


In [28]:
lsdf = []
for i in range(len(dfPDF)):
    row = dfPDF.iloc[i]
    if row['RESPONSE'] is None:
        continue
    for prod in row['RESPONSE']:
        df = pd.DataFrame({'FULL_PATH':[row['FULL_PATH']],
                           'FILE_NAME':[row['FILE_NAME']],
                           'PRODUCT':[prod['PRODUCT_NAME']],
                           'SUPPLIER':[prod['SUPPLIER_NAME']]})
        lsdf.append(df)
dfPROD = pd.concat(lsdf, ignore_index=True)
dfPROD.head()

,FULL_PATH,FILE_NAME,PRODUCT,SUPPLIER
0,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003031-Actimalt Liquid Regular.pdf,ACTIMALT LIQUID REGULAR,Muntons
1,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003047-8. FAMB279 BROWN SUGAR TYPE FL.pdf,FAMB279 N&A BROWN SUGAR TYPE FL,"WILD Flavors, Inc."
2,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003729-4. Acerola JC_TH.pdf,"Acerola juice concentrate, clarified R=64-68",SVZ Tomaszow Sp. z o.o.
3,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003890-ACCELERZYME CPG.pdf,Accelerzyme CPG BF,DSM Food Specialties B.V.
4,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000007436-Specification_1101103010101_Apple...,Apple juice concentrate,AUSTRIA JUICE GmbH


In [33]:
dfPROD['RESPONSE'] = None
for i in range(len(dfPROD)):
    try:
        row = dfPROD.iloc[i]
        full_path = row['FULL_PATH']
        file_name = row['FILE_NAME']
        url = "http://127.0.0.1:8000/v1_parse_pim_fields"
        data = {
            "inputProductName": row['PRODUCT'],
            "inputBusinessLine": businessLine,
            "inputSecret": os.getenv('CUSTOM_SECRET1'),
            # booleans sent as strings in multipart forms
            "inputWebSearch": "false",
            "inputParallel": "true"}
        files = [("inputListDocumentation", ("PIM00000671-TDS.pdf", open(row["FULL_PATH"], "rb"), "application/pdf"))]
        response = requests.post(url, data=data, files=files, timeout=300)
        dfPROD['RESPONSE'].iat[i] = response.json()
        print('SUCCESS ', file_name)
    except:
        print('FAILURE ', file_name)

SUCCESS  PIM000003031-Actimalt Liquid Regular.pdf
SUCCESS  PIM000003047-8. FAMB279 BROWN SUGAR TYPE FL.pdf
SUCCESS  PIM000003729-4. Acerola JC_TH.pdf
SUCCESS  PIM000003890-ACCELERZYME CPG.pdf
SUCCESS  PIM000007436-Specification_1101103010101_Apple juice concentrate_Austria Juice Standard - Stock quality - Medium acidity.pdf
SUCCESS  PIM000008862-EXTER 100 TDS 2021.pdf
SUCCESS  PIM000009406-Rokoagar LS ORGANIC - Food Applications Eng - E01.pdf
SUCCESS  PIM000009441-SPEC APPLE FLAVOUR NPC1917M.pdf
SUCCESS  PIM000009983-YU15 rev02 - organic chickpeas flour.pdf
SUCCESS  PIM000021685-Specification Apple JC 35209000710000 20180717.pdf
SUCCESS  PIM000039886-973772999 DPS Apple Pomace fine milled .pdf
SUCCESS  PIM000041915-K110 rev08 - Q.C. Organic kamut.pdf
SUCCESS  PIM000042350-Spec-AD Flat Parsley Medium, Large.pdf
SUCCESS  PIM000042350-Spec-AD Flat Parsley Medium, Large.pdf
SUCCESS  PIM000042350-Spec-AD Flat Parsley Medium, Large.pdf
SUCCESS  PIM000042350-Spec-AD Flat Parsley Medium, Large

In [37]:
dfPROD

,FULL_PATH,FILE_NAME,PRODUCT,SUPPLIER,RESPONSE
0,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003031-Actimalt Liquid Regular.pdf,ACTIMALT LIQUID REGULAR,Muntons,{'inputProductName': 'ACTIMALT LIQUID REGULAR'...
1,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003047-8. FAMB279 BROWN SUGAR TYPE FL.pdf,FAMB279 N&A BROWN SUGAR TYPE FL,"WILD Flavors, Inc.",{'inputProductName': 'FAMB279 N&A BROWN SUGAR ...
2,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003729-4. Acerola JC_TH.pdf,"Acerola juice concentrate, clarified R=64-68",SVZ Tomaszow Sp. z o.o.,{'inputProductName': 'Acerola juice concentrat...
3,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000003890-ACCELERZYME CPG.pdf,Accelerzyme CPG BF,DSM Food Specialties B.V.,"{'inputProductName': 'Accelerzyme CPG BF', 'in..."
4,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000007436-Specification_1101103010101_Apple...,Apple juice concentrate,AUSTRIA JUICE GmbH,{'inputProductName': 'Apple juice concentrate'...
5,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000008862-EXTER 100 TDS 2021.pdf,EXTER 100 - Bouillon,Exter B.V.,"{'inputProductName': 'EXTER 100 - Bouillon', '..."
6,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000009406-Rokoagar LS ORGANIC - Food Applic...,ROKOAGAR LS ORGANIC,"INDUSTRIAS ROKO, S.A.","{'inputProductName': 'ROKOAGAR LS ORGANIC', 'i..."
7,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000009441-SPEC APPLE FLAVOUR NPC1917M.pdf,APPLE FLAVOUR NPC1917M,Matrix Flavours & Fragrances Sdn Bhd,"{'inputProductName': 'APPLE FLAVOUR NPC1917M',..."
8,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000009983-YU15 rev02 - organic chickpeas fl...,Pre-gelatinized organic chickpeas flour,Naturis,{'inputProductName': 'Pre-gelatinized organic ...
9,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...,PIM000021685-Specification Apple JC 3520900071...,Apple Juice Concentrate,WILD,{'inputProductName': 'Apple Juice Concentrate'...


In [ ]:
dfPROD

In [1]:
i=0

row = dfPROD.iloc[i]
row['RESPONSE']

NameError: name 'dfPROD' is not defined

In [32]:
response.json()

{'inputProductName': 'ACTIMALT LIQUID REGULAR',
 'inputBusinessLine': 'FBI',
 'inputListDocumentation': [{'filename': 'PIM00000671-TDS.pdf',
   'file': {},
   'size': 101146,
   'headers': {'content-disposition': 'form-data; name="inputListDocumentation"; filename="PIM00000671-TDS.pdf"',
    'content-type': 'application/pdf'},
   '_max_mem_size': 1048576}],
 'inputSecret': 'HIDDEN',
 'inputWebSearch': False,
 'inputParallel': True,
 'stg_businessLineStr': 'Food & Beverage',
 'stg_lsTempFile': 'HIDDEN',
 'stg_lsParsedText': 'HIDDEN',
 'stg_parsedText': 'HIDDEN',
 'stg_lsBase64': 'HIDDEN',
 'gpt_manufacturer_or_supplier_answer': 'Muntons',
 'gpt_manufacturer_or_supplier_reason': "The product specification document for ACTIMALT LIQUID REGULAR prominently features the name 'Muntons' at the top, indicating that Muntons is the manufacturer or supplier of this product.",
 'gpt_composition_search_answer': '',
 'gpt_function_search_answer': '',
 'gpt_application_search_answer': '',
 'gpt_combin

In [ ]:
for i in range(len(dfPDF)):
    row = 

    df = parsePDF_AzureAI(dfPDF['FULL_PATH'][i])
    print(df.head(2))

In [16]:
response.json()['products_and_suppliers']

[{'PRODUCT_NAME': 'ACTIMALT LIQUID REGULAR', 'SUPPLIER_NAME': 'Muntons'}]

In [ ]:
for i in range(len(dfPDF)):
    print(dfPDF['FULL_PATH'][i])
    df = parsePDF_AzureAI(dfPDF['FULL_PATH'][i])
    print(df.head(2))

,FULL_PATH
0,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
1,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
2,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
3,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
4,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
5,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
6,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
7,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
8,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...
9,versionAchive/V0.9-alpha/testFiles/FBI/PIM0000...


In [8]:
lspdf

['PIM000003031-Actimalt Liquid Regular.pdf',
 'PIM000003047-8. FAMB279 BROWN SUGAR TYPE FL.pdf',
 'PIM000003729-4. Acerola JC_TH.pdf',
 'PIM000003890-ACCELERZYME CPG.pdf',
 'PIM000007436-Specification_1101103010101_Apple juice concentrate_Austria Juice Standard - Stock quality - Medium acidity.pdf',
 'PIM000008862-EXTER 100 TDS 2021.pdf',
 'PIM000009406-Rokoagar LS ORGANIC - Food Applications Eng - E01.pdf',
 'PIM000009441-SPEC APPLE FLAVOUR NPC1917M.pdf',
 'PIM000009983-YU15 rev02 - organic chickpeas flour.pdf',
 'PIM000021685-Specification Apple JC 35209000710000 20180717.pdf',
 'PIM000039886-973772999 DPS Apple Pomace fine milled .pdf',
 'PIM000041915-K110 rev08 - Q.C. Organic kamut.pdf',
 'PIM000042350-Spec-AD Flat Parsley Medium, Large.pdf',
 'PIM000047605-702469-001(20221018)FD Cranberry Powder 50mesh-DKSH AU.pdf',
 'PIM000048276-RSF-200 - Prod Info rev. 3 (1).pdf',
 'PIM000088058-LACIN101-E-202101.pdf',
 'PIM000088545-Specification-Rosemary Extract 5%.pdf',
 'PIM000088817-Produc

In [5]:
import pandas as pd